In [34]:
import pandas as pd
import numpy as np

### 1번

In [35]:
df_con = pd.read_csv('../데이터자료/1. 상장폐지종목 정리끝.csv', encoding='cp949', index_col=0)
df_invis = pd.read_csv('../데이터자료/1. 코스닥 기업 개별재무제표.csv', encoding='cp949')


FileNotFoundError: [Errno 2] No such file or directory: '../데이터자료/1. 상장폐지종목 정리끝.csv'

In [ ]:
df_con.columns

In [ ]:
df_con = df_con[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명','[제조]매출액증가율(IFRS연결)',
'[제조]자기자본순이익률(IFRS연결)','[제조]매출액순이익률(IFRS연결)','[제조]유동비율(IFRS연결)','[제조]자기자본구성비율(IFRS연결)',
'[제조]총자본회전률(IFRS연결)','[제조]자기자본회전률(IFRS연결)','[제조]총자본투자효율(IFRS연결)','[제조]설비투자효율(IFRS연결)',
'[제조]순이익증가율(IFRS연결)']]

In [ ]:
df_invis.columns

In [ ]:
df_invis=df_invis[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명','[제조]매출액증가율(IFRS)',
'[제조]자기자본순이익률(IFRS)','[제조]매출액순이익률(IFRS)','[제조]유동비율(IFRS)','[제조]자기자본구성비율(IFRS)',
'[제조]총자본회전률(IFRS)','[제조]자기자본회전률(IFRS)','[제조]총자본투자효율(IFRS)','[제조]설비투자효율(IFRS)',
'[제조]순이익증가율(IFRS)']]

In [ ]:

# 데이터 기본 정보 확인
df_con.info()
print(df_con.head(20))
df_invis.info()
print(df_invis.head(20))

# 컬럼명 일치를 위한 컬럼명 변경
df_con.columns = df_con.columns.str.replace('연결', '')

# 누락된 데이터 채우기
fill_columns = df_con.columns[df_con.columns.str.contains("\[제조\]")]
for col in fill_columns:
    df_con[col] = df_con[col].fillna(df_invis[col])

# 누락된 데이터가 있는 행 제거
df_con.dropna(subset=fill_columns, inplace=True)

# 상장폐지일이 존재하면서 소속코드가 5인 데이터 처리
df_con.loc[(~df_con["상장폐지일"].isnull()) & (df_con["소속코드"] == 5), "상장폐지일"] = np.NaN

# 컬럼명 간소화
df_con.columns = df_con.columns.str.replace('[\[제조\]\(\)IFRS]', '', regex=True)

# 최종 데이터 저장
df_con.to_csv('../데이터자료/2. 연결개별통합_리팩토링.csv', index=False)

### 2번

In [ ]:
import pandas as pd
import numpy as np
# 데이터 로드
df_1 = pd.read_csv('../데이터자료/0. 연결재무제표.csv', encoding='cp949')
df_2 = pd.read_csv('../데이터자료/0. 코스닥기업 연결재무제표.csv', encoding='cp949')


In [ ]:
df_1=df_1[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명','[제조]매출액증가율(IFRS연결)',
'[제조]자기자본순이익률(IFRS연결)','[제조]매출액순이익률(IFRS연결)','[제조]유동비율(IFRS연결)','[제조]자기자본구성비율(IFRS연결)',
'[제조]총자본회전률(IFRS연결)','[제조]자기자본회전률(IFRS연결)','[제조]총자본투자효율(IFRS연결)','[제조]설비투자효율(IFRS연결)',
'[제조]순이익증가율(IFRS연결)']]

In [ ]:
df_2=df_2[['회사명', '거래소코드', '회계년도', '[제조]매출액증가율(IFRS연결)',
'[제조]자기자본순이익률(IFRS연결)','[제조]매출액순이익률(IFRS연결)','[제조]유동비율(IFRS연결)','[제조]자기자본구성비율(IFRS연결)',
'[제조]총자본회전률(IFRS연결)','[제조]자기자본회전률(IFRS연결)','[제조]총자본투자효율(IFRS연결)','[제조]설비투자효율(IFRS연결)',
'[제조]순이익증가율(IFRS연결)']]

In [ ]:



# 데이터 기본 정보 확인
df_2.info()
df_1.info()

# 상장폐지된 데이터 필터링
df_default = df_1[~df_1["상장폐지일"].isnull()]
print("상장폐지 기업 수:", len(df_default["회사명"].unique()))

# 2022년도 상장폐지 데이터 처리
df_default_2022 = df_default[df_default["상장폐지일"].str.contains('2022', na=False)]
for idx in df_default_2022.index:
    df_default.loc[idx, "상장폐지일"] = np.NaN
    df_default.loc[idx, "소속코드"] = 5

# 상장폐지 사유 데이터 처리
df_default_mod = pd.read_csv('../데이터자료/1. 상장폐지종목_사유등록.csv', encoding='cp949', index_col=0)
df = pd.concat([df_1, df_default_mod["상장폐지사유"]], axis=1)

# 코넥스, 합병 관련 데이터 제거
df = df[~df["상장폐지사유"].str.contains('코넥스', na=False)]
df = df[~df["상장폐지사유"].str.contains('합병', na=False)]

# 상장폐지 사유가 있는 데이터의 소속코드를 5로 수정
df.loc[~df["상장폐지사유"].isnull(), "소속코드"] = 5

# 날짜 데이터 처리 및 필터링
df["상장일"] = pd.to_datetime(df['상장일'])
df["상장폐지일"] = pd.to_datetime(df['상장폐지일'])
df["회계년도"] = pd.to_datetime(df["회계년도"])
df = df[df["상장폐지일"].dt.year >= 2011]

# 회계년도 기준 필터링
df["회계월"] = df["회계년도"].dt.month
df = df[df["회계월"] == 12]

# 외국계 기업 제거
df = df[df["거래소코드"] <= 900000]

# 중복 데이터 확인 및 제거
df["상장년도"] = df["상장일"].dt.year
df["회계년"] = df["회계년도"].dt.year
df["연차"] = df["회계년"] - df["상장년도"]
df = df[df["연차"] > 0]

b = df[["회사명", "거래소코드"]]
d = b.groupby("회사명")["거래소코드"].unique()
for company, codes in d.items():
    if len(codes) > 1:
        df = df[~df["거래소코드"].isin(codes)]

# 인덱스 초기화 및 최종 데이터 저장
df.reset_index(drop=True, inplace=True)
df.to_csv('../데이터자료/상장폐지종목 정리끝1.csv', encoding='cp949')

### 파생변수

In [44]:
df = pd.read_csv('../데이터자료/5. label 지정 완료.csv',encoding="cp949")

In [45]:
# 변수 10개
df=df[['회사명', '거래소코드', '회계년도', '매출액증가율',
'자기자본순이익률','매출액순이익률','유동비율','자기자본구성비율',
'총자본회전률','자기자본회전률','총자본투자효율','설비투자효율',
'순이익증가율']]

In [46]:
df.to_csv('변수10개.csv')

In [43]:
import pandas as pd
from scipy import stats

# 파일 불러오기 및 기본 구조 확인
# file_path = 'cleaned_111.csv' 
# df = pd.read_csv(file_path)
print(df.head()) # 데이터의 처음 몇 행을 출력

# 기본 정보 및 결측치 확인
print(df.info()) # 데이터의 기본 정보 출력
print(df.isnull().sum()) # 각 컬럼별 결측치 수 확인

# 회사별 재무 상태 변화 분석
# 데이터를 회사명과 회계년도로 그룹화하여 평균값 계산
df_grouped = df.groupby(['회사명', '회계년도']).mean()
print(df_grouped.head()) # 결과 확인

# t-test 수행
# 2010년과 2019년 데이터를 각각 분리
df_2010 = df[df['회계년도'] == '2010/12']
df_2019 = df[df['회계년도'] == '2019/12']

# 파생변수 생성
# 데이터를 회사명으로 정렬
df_sorted = df.sort_values(['회사명', '회계년도'])

# '매출액순이익률'의 연도별 차이와 비율을 계산
df_sorted['매출액순이익률_diff'] = df_sorted.groupby('회사명')['매출액순이익률'].diff()
df_sorted['매출액순이익률_ratio'] = df_sorted.groupby('회사명')['매출액순이익률'].pct_change()
print(df_sorted[['회사명', '회계년도', '매출액순이익률_diff', '매출액순이익률_ratio']].head()) # 결과 확인

# 상위 10개 기업 선정 및 Gap Model 변수 생성
# '매출액순이익률' 기준 상위 10개 기업 추출
top_10_companies = df_sorted.groupby('회사명')['매출액순이익률'].mean().sort_values(ascending=False).head(10)

# 상위 10개 기업의 평균 계산
top_10_average = top_10_companies.mean()

# 산업 전체 평균 계산
industry_average = df_sorted['매출액순이익률'].mean()

# Gap Model 변수 생성
df_sorted['Top_10_Gap_Diff'] = df_sorted['매출액순이익률'] - top_10_average
df_sorted['Top_10_Gap_Ratio'] = df_sorted['매출액순이익률'] / top_10_average
df_sorted['Average_Gap_Diff'] = df_sorted['매출액순이익률'] - industry_average
df_sorted['Average_Gap_Ratio'] = df_sorted['매출액순이익률'] / industry_average
print(df_sorted[['회사명', '회계년도', '매출액순이익률', 'Top_10_Gap_Diff', 'Top_10_Gap_Ratio', 
                 'Average_Gap_Diff', 'Average_Gap_Ratio']].head()) # 결과 확인

        회사명  거래소코드        회계년도  매출액증가율  자기자본순이익률  매출액순이익률    유동비율  자기자본구성비율  \
0  (주)CMG제약  58820  2011-12-01   56.40     -8.28    -8.50  200.97     66.05   
1  (주)CMG제약  58820  2012-12-01   -8.43    -26.84   -40.97  693.49     55.32   
2  (주)CMG제약  58820  2013-12-01   25.51      0.74     1.45  694.71     87.63   
3  (주)CMG제약  58820  2014-12-01   11.52      0.32     0.69  324.58     84.39   
4  (주)CMG제약  58820  2015-12-01   18.85     -7.62   -13.50  296.85     82.80   

   총자본회전률  자기자본회전률  총자본투자효율  설비투자효율  순이익증가율  
0    0.66     0.97    12.72   35.86    0.00  
1    0.39     0.66     0.47    2.89    0.00  
2    0.36     0.51    12.51   77.91    0.00  
3    0.40     0.46    12.16   37.80  -46.64  
4    0.47     0.56     7.63   24.21    0.00  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9362 entries, 0 to 9361
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   회사명       9362 non-null   object 
 1   거래소코드     9362 non-n

In [39]:
df_sorted.head(10)

,회사명,거래소코드,회계년도,매출액증가율,자기자본순이익률,매출액순이익률,유동비율,자기자본구성비율,총자본회전률,자기자본회전률,총자본투자효율,설비투자효율,순이익증가율,매출액순이익률_diff,매출액순이익률_ratio,Top_10_Gap_Diff,Top_10_Gap_Ratio,Average_Gap_Diff,Average_Gap_Ratio
0,(주)CMG제약,58820,2011-12-01,56.40,-8.28,-8.50,200.97,66.05,0.66,0.97,12.72,35.86,0.00,NaN,NaN,-147.475436,-0.061162,12.804023,0.398986
1,(주)CMG제약,58820,2012-12-01,-8.43,-26.84,-40.97,693.49,55.32,0.39,0.66,0.47,2.89,0.00,-32.47,3.820000,-179.945436,-0.294800,-19.665977,1.923111
2,(주)CMG제약,58820,2013-12-01,25.51,0.74,1.45,694.71,87.63,0.36,0.51,12.51,77.91,0.00,42.42,-1.035392,-137.525436,0.010433,22.754023,-0.068062
3,(주)CMG제약,58820,2014-12-01,11.52,0.32,0.69,324.58,84.39,0.40,0.46,12.16,37.80,-46.64,-0.76,-0.524138,-138.285436,0.004965,21.994023,-0.032388
4,(주)CMG제약,58820,2015-12-01,18.85,-7.62,-13.50,296.85,82.80,0.47,0.56,7.63,24.21,0.00,-14.19,-20.565217,-152.475436,-0.097139,7.804023,0.633683
5,(주)CMG제약,58820,2016-12-01,21.92,2.34,5.24,862.15,91.10,0.39,0.45,9.48,62.31,0.00,18.74,-1.388148,-133.735436,0.037705,26.544023,-0.245963
6,(주)CMG제약,58820,2017-12-01,25.98,2.85,6.83,746.28,89.39,0.38,0.42,12.51,84.87,64.07,1.59,0.303435,-132.145436,0.049145,28.134023,-0.320597
7,(주)CMG제약,58820,2018-12-01,20.05,6.01,12.16,957.86,90.91,0.45,0.49,14.73,106.25,113.94,5.33,0.780381,-126.815436,0.087497,33.464023,-0.570784
8,(주)CMG제약,58820,2019-12-01,17.56,2.15,5.20,1618.07,93.71,0.38,0.41,9.90,114.60,-49.69,-6.96,-0.572368,-133.775436,0.037417,26.504023,-0.244085
9,(주)CMG제약,58820,2020-12-01,9.96,0.12,0.33,1069.54,92.72,0.33,0.36,8.71,108.22,-92.95,-4.87,-0.936538,-138.645436,0.002375,21.634023,-0.015490


In [ ]:
# 파일을 다시 불러오기
df1_updated = pd.read_csv('kosdaq_2012_1parm.csv', encoding='cp949')
df2_updated = pd.read_csv('kosdaq_2012_1parm_live.csv', encoding='cp949')

# 각 파일에서 회사명만 추출
companies_1_updated = set(df1_updated['회사명'])
companies_2_updated = set(df2_updated['회사명'])

# 첫 번째 파일에만 있는 회사명 추출
unique_companies_in_1_not_in_2_updated = companies_1_updated - companies_2_updated

# 결과 출력
unique_companies_in_1_not_in_2_updated

In [ ]:
# 각 CSV 파일에서 '회사명'이 '(주)맘스터치앤컴퍼니'인 행 찾기
moms_touch_and_company_df1 = df1_updated[df1_updated['회사명'] == '(주)맘스터치앤컴퍼니']
moms_touch_and_company_df2 = df2_updated[df2_updated['회사명'] == '(주)맘스터치앤컴퍼니']

# 결과 확인
moms_touch_and_company_df1, moms_touch_and_company_df2